[DIY Covid-19 Dashboard](https://github.com/3miki/diy-covid-19-dashboard) (C) Miki Suzuki, 2023. This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [3]:
data_filename = 'AnalysisByAge220101.json'
with open(data_filename, 'rt') as data_file:
    analysis_by_age_initial_data=json.load(data_file)

In [10]:
def min_age(agerange):
    agerange = agerange.replace('+','')
    start = agerange.split('_')[0]
    return start
 
# The First graph uses data from the two metrics 'newCasesBySpecimenDateAgeDemographics' and 'newReinfectionsBySpecimenDateAgeDemographics'.
def wrangle_data1(apidata):
    data_dict = apidata['data'][0]
    case = data_dict['CaseByAge']
    reinfection = data_dict['ReinfectionByAge']
    ageranges1 = [x['age'] for x in case]
    # Remove age ranges unwanted in the graph. The age range '00-59' and '60+' are total numbers within the age ranges (duplicate).
    ageranges1.remove('00_59')
    ageranges1.remove('60+')
    ageranges1.remove('unassigned')
    ageranges1.sort(key=min_age) 
    age_df1 = pd.DataFrame(index=ageranges1, columns=['Total Infection','Reinfection'])
    for entry in case:
        if entry['age'] in ageranges1:
            ageband = entry['age']
            age_df1.loc[ageband, 'Total Infection']=entry['cases'] if entry['cases'] != None else 0.0
    for entry in reinfection:
        if entry['age'] in ageranges1:
            ageband = entry['age']
            age_df1.loc[ageband, 'Reinfection']=entry['cases'] if entry['cases'] != None else 0.0
    return age_df1

# The metric 'vaccinationsAgeDemographics' has different age groups from the above metrics and needs a separate graph.
# The second graph uses data from this metric which is divided into three categories: 'First Dose', 'Second Dose', and 'Third Dose'.
def wrangle_data2(apidata):
    data_dict = apidata['data'][0]
    vaccination = data_dict['VaccinationByAge']    
    ageranges2 = [x['age'] for x in vaccination]
    ageranges2.remove('50+')
    ageranges2.remove('65+')
    ageranges2.remove('75+')
    ageranges2.sort(key=min_age)
    age_df2 = pd.DataFrame(index=ageranges2, columns=['First Dose', 'Second Dose', 'Third Dose'])
    for entry in vaccination:
        if entry['age'] in ageranges2:
            ageband = entry['age']
            age_df2.loc[ageband, 'First Dose'] = entry['cumPeopleVaccinatedFirstDoseByVaccinationDate'] if entry['cumPeopleVaccinatedFirstDoseByVaccinationDate'] != None else 0.0
            age_df2.loc[ageband, 'Second Dose'] = entry['cumPeopleVaccinatedSecondDoseByVaccinationDate'] if entry['cumPeopleVaccinatedSecondDoseByVaccinationDate'] != None else 0.0
            age_df2.loc[ageband, 'Third Dose'] = entry['cumPeopleVaccinatedThirdInjectionByVaccinationDate'] if entry['cumPeopleVaccinatedThirdInjectionByVaccinationDate'] != None else 0.0
    return age_df2

age_df1 = wrangle_data1(analysis_by_age_initial_data)
age_df2 = wrangle_data2(analysis_by_age_initial_data)

In [5]:
def access_api(date: str):
    # The data was too heavy to load if the date was not specified.   
    # Date can be selected from options in a dropdown widget. 
    filters = [
        'areaType=nation',
        'areaName=England',
        f'date={date}'
    ]
    structure = {
        'CaseByAge':'newCasesBySpecimenDateAgeDemographics',
        'ReinfectionByAge':'newReinfectionsBySpecimenDateAgeDemographics',
        'VaccinationByAge':'vaccinationsAgeDemographics'
    }
    
    api = Cov19API(filters=filters, structure=structure)
    analysisByAge = api.get_json()

    # A new JSON file will be created for the selected date.
    # If the file already exists, it will be overwritten.
    covid_filename = f'AnalysisByAge_{date}.json'
    with open(covid_filename, "wt") as file:
        json.dump(analysisByAge, file)
    return analysisByAge

## Pick any date from the following options to display graphs of your selected date

The initial specified date for the following graphs is 2022-01-01. 
You can choose a date from the options below.

In [6]:
# In addition to refreshing data, the following button can fetch the data of a selected date from the options. 
select_date = wdg.Dropdown(
    options=['2021-01-01', '2021-07-01', '2022-01-01', '2022-07-01', '2023-01-01', '2023-07-01'],
    value='2022-01-01',
    description='Select a date',
    disabled=False,
)

def api_button_callback(button):
    global age_df1  
    global age_df2
    print('Starting refresh API call')   # Monitor an API call status in a log.
    # While loading new data, the button is disabled with the spinner icon and 'Loading' text.
    api_button.disabled = True
    api_button.description = 'Loading...'
    api_button.icon = 'spinner'
    try:
        new_api_data=access_api(select_date.value)
        # Wrangle the data and overwrite the two dataframes for plotting.          
        wrangle_data1(new_api_data).plot()
        age_df1=wrangle_data1(new_api_data)
        wrangle_data2(new_api_data).plot()
        age_df2=wrangle_data2(new_api_data)
        refresh_graphs() # Two graphs will be refreshed with this function.
        print('Finished refresh API call') 
        # Change the button back to the original state (status and description) but show the check icon instead
        api_button.disabled = False
        api_button.description = 'Change date'       
        api_button.icon = 'check'
    except Exception as e:
        print(f'Error message: {e}')
        api_button.button_style = 'warning'
        api_button.description = 'Error occurred'        
        api_button.icon = 'exclamation-triangle'

api_button = wdg.Button(
    description='Change date',
    disabled=False,
    button_style='success',
    tooltip='Click me to refresh graphs',
    icon='download'
)

api_button.on_click(api_button_callback)
controls = wdg.HBox([select_date, api_button])
display(controls)

In [7]:
def refresh_graphs():
    # Save the current values, change them by setting the values with empty lists, 
    # and then set them back to the current values.
    current = infection_columns.value
    infection_columns.value = []
    infection_columns.value = current
    current = vaccine_columns.value
    vaccine_columns.value = []
    vaccine_columns.value = current
    # Update the two graphs according to the new data of the selected date.
    # The widgets need to be refreshed as well.
    graph1 = wdg.interactive_output(age_graph1, {'graphcolumns1': infection_columns})    
    controls1 = wdg.HBox([infection_columns, select_all_button1])
    display(controls1, graph1)
    graph2 = wdg.interactive_output(age_graph2, {'graphcolumns2': vaccine_columns})
    controls2 = wdg.HBox([vaccine_columns, select_all_button2])
    display(controls2, graph2)

## Total cases and reinfections over different age groups in England (on the selected date)

The graph below displays two metrics: "newCasesBySpecimenDateAgeDemographics" as Total Infection, "newReinfectionsBySpecimenDateAgeDemographics" as Reinfection. 

You can display each data separately by selecting the options below or show them all again by clicking 'Select all'.

In [8]:
def age_graph1(graphcolumns1):
    ncolumns1 = len(graphcolumns1)
    if ncolumns1>0:
        age_df1.plot.bar(y=list(graphcolumns1), color=['lightgreen', 'orange'])
        plt.xticks(rotation=50, horizontalalignment="center")
        plt.title('Total cases and reinfections over different age groups in England')
        plt.xlabel('Age Group')
        plt.ylabel('Number of Cases')
        plt.show()
    else:
        print('Click above selections to display a graph')

# Create a multiple-selection widget for the first graph.
infection_columns = wdg.SelectMultiple(
    options=['Total Infection','Reinfection'],
    value=['Total Infection','Reinfection'],
    rows=2,
    description='New Cases:',
    disabled=False
)

graph1 = wdg.interactive_output(age_graph1, {'graphcolumns1': infection_columns})

# Create a 'Select all' button widget because it is not clear how to select multiple options (do shift + click)
def select_all_button_callback1(*args):
    infection_columns.value = infection_columns.options

select_all_button1 = wdg.Button(
    description = 'Select all',
    tooltip='Click me to select all options'
)

select_all_button1.on_click(select_all_button_callback1)
controls1 = wdg.HBox([infection_columns, select_all_button1])
display(controls1, graph1)

Output()

## Number of people who received vaccinations over different age groups in England

The graph below displays a metric "vaccinationsAgeDemographics" with three categories: "cumPeopleVaccinatedFirstDoseByVaccinationDate" as First Dose, "cumPeopleVaccinatedSecondDoseByVaccinationDate" as Second Dose, and "cumPeopleVaccinatedThirdInjectionByVaccinationDate" as Third Dose. 


You can display each data separately by selecting the options below or show them all again by clicking 'Select all'.

In [9]:
def age_graph2(graphcolumns2):
    ncolumns2 = len(graphcolumns2)
    if ncolumns2>0:
        age_df2.plot(kind='bar', y=list(graphcolumns2), color=['orange', 'lightgreen', '#0AA7CE'])
        plt.xticks(rotation=50, horizontalalignment="center")
        plt.title('Number of people who received vaccinations over different age groups in England')
        plt.xlabel('Age Group')
        plt.ylabel('Number of Cases')
        plt.show()
    else:
        print('click above selections to refresh graph')

vaccine_columns = wdg.SelectMultiple(
    options=['First Dose', 'Second Dose', 'Third Dose'],
    value=['First Dose', 'Second Dose', 'Third Dose'],
    rows=3,
    description='Vaccination:',
    disabled=False
)

graph2 = wdg.interactive_output(age_graph2, {'graphcolumns2': vaccine_columns})

def select_all_button_callback2(*args):
    vaccine_columns.value = vaccine_columns.options
    
select_all_button2 = wdg.Button(
    description = 'Select all',
    tooltip='Click me to select all options'
)

select_all_button2.on_click(select_all_button_callback2)
controls2 = wdg.HBox([vaccine_columns, select_all_button2])
display(controls2, graph2)

Output()

**Author and Copyright Notice** Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi, Copyright (C) Miki Suzuki 2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).